# Explore PUDL FERC Form 6 Data

In [1]:
import sqlite3
import pandas as pd

In [2]:
# Set pandas options
pd.set_option('display.max_columns', None)

## Purpose

The purpose of this notebook will be to explore how to use Python to extract [(FERC)](https://ferc.gov/what-ferc) Form 6 data provided by the Public Utility Data Liberation [(PUDL)](https://catalyst.coop/pudl/) Project.

The goal will be to extract Income Statement data for Montana oil pipelines from the PUDL Sqlite database and match to the FERC Form 6 exported from the FERC website.

The following oil pipelines operate in Montna and file a FERC Form 6:
- Bridger Pipeline LLC
- Cenex Pipeline, LLC
- Express Pipeline LLC
- Front Range Pipeline, LLC
- Hiland Crude, LLC
- North Dakota Pipeline Company LLC
- ONEOK Bakken Pipeline, L.L.C.
- Oneok Elk Creek Pipeline, L.L.C.
- Par Rocky Mountain Midstream, LLC
- Phillips 66 Carrier LLC
- Phillips 66 Pipeline LLC
- Red Butte Pipe Line LLC
- Rocky Mountain Pipeline Montana LLC
- Tesoro High Plains Pipeline Company LLC
- Yellowstone Pipe Line Company

## Establish Connection to PUDL Database

The first step will be to connect to the SQLite database downloaded from the PUDL website.

In [3]:
# Connect to the database
conn = sqlite3.connect("ferc6_xbrl.sqlite")

In [4]:
# Create a cursor that lets you execute SQL queries and fetch results
cursor = conn.cursor()

In [5]:
# Get list of table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

## Explore Tables Contained in the Database

The PUDL SQLite database conatins many tables, each representing various schedules and statements. With the database loaded we can now extract a list of the various tables available.

In [6]:
# Assign list of tables to variable
tables = cursor.fetchall()

In [7]:
# Print list of table names
print("Tables in the database:")
for table in tables:
    print(table[0])

Tables in the database:
annual_corporate_officer_certification_001_duration
identification_001_duration
list_of_schedule_002_duration
general_information_101_duration
control_over_respondent_102_duration
control_over_respondent_102_instant
companies_controlled_by_respondent_103_duration
companies_controlled_by_respondent_103_instant
principal_general_officers_104_duration
directors_105_duration
important_changes_during_the_quarter_year_108_duration
comparative_balance_sheet_110_instant
income_statement_114_duration
statement_of_accumulated_other_comprehensive_income_and_hedging_activities_116_duration
statement_of_accumulated_other_comprehensive_income_and_hedging_activities_116_instant
appropriated_retained_income_118_instant
appropriated_retained_income_other_appropriations_118_duration
appropriated_retained_income_other_appropriations_118_instant
unappropriated_retained_income_statement_119_duration
unappropriated_retained_income_statement_119_instant
statement_of_cash_flows_120_dur

## Extract Income Statement Data

Below are screenshots of the FERC Form 6 Income Statement for Yellowstone Pipe Line Company:

![](img/ypl-income-statement-image-01.png)

![](img/ypl-income-statement-image-02.png)

![](img/ypl-income-statement-image-03.png)

![](img/ypl-income-statement-image-04.png)

By looking at the list of tables from earlier, "income_statement_114_duration" would appear to be the relevant table name as the FERC Form 6, Income Statement is on Page 114 of the form. To verify this we can assign this table to a Pandas DataFrame to explore the table columns.

In [8]:
# Assign income statement table to a DataFrame then close the sql connection
df_is = pd.read_sql("SELECT * FROM income_statement_114_duration", conn)
conn.close()

In [9]:
df_is.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2369 entries, 0 to 2368
Data columns (total 34 columns):
 #   Column                                                                              Non-Null Count  Dtype  
---  ------                                                                              --------------  -----  
 0   entity_id                                                                           2369 non-null   object 
 1   filing_name                                                                         2369 non-null   object 
 2   publication_time                                                                    2369 non-null   object 
 3   start_date                                                                          2369 non-null   object 
 4   end_date                                                                            2369 non-null   object 
 5   provision_for_deferred_taxes_extraordinary_items                                    312 non-null 

The initial DataFrame contains information for 2,000+ filers. For this example, we will start with pulling information for Yellowstone Pipe Line Company (YPL). To be able to do this the DataFrame will need to be filtered. Looking at the above it appears column **entity_id** contains a unique identifier for each filer and **filing_name** contains the name of the filer. The **filing_name** column will first be used to determine which **entity_id** corresponds to YPL.

In [10]:
# Use str.contains to find Yellowstone Pipe Line's entity id
df_is[df_is.loc[:, 'filing_name'].str.contains('Yellowstone')]

,entity_id,filing_name,publication_time,start_date,end_date,provision_for_deferred_taxes_extraordinary_items,extraordinary_items_net,equity_in_earnings_losses_of_affiliated_companies_including_dividend_income,net_income_loss,income_taxes_on_extraordinary_items,interest_expense,gain_loss_from_disposition_of_discontinued_segments_less_applicable_income_taxes,extraordinary_items_and_accounting_changes,provision_for_deferred_taxes,income_loss_from_operations_of_discontinued_segments_less_applicable_income_taxes,dividend_income_equity_investments,operating_expenses,undistributed_earnings_losses,income_loss_from_continuing_operations,miscellaneous_income,ordinary_income_before_federal_income_taxes,cumulative_effect_of_changes_in_accounting_principles_less_applicable_income_taxes,unusual_or_infrequent_items_credit,income_loss_from_discontinued_operations,federal_income_taxes_on_income_from_continuing_operations,net_carrier_operating_income,miscellaneous_income_charges,income_net_from_noncarrier_property,unusual_or_infrequent_items_debit,operating_revenues,other_income_and_deductions,interest_and_dividend_income,extraordinary_items,income_loss_before_extraordinary_items
326,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1650134001,2022-04-16 18:33:21.000000,2021-01-01,2021-12-31,NaN,NaN,NaN,10446634.0,NaN,6780.0,NaN,NaN,133175.0,NaN,NaN,26415762.0,NaN,10446634.0,2565.0,14734177.0,NaN,NaN,NaN,4154368.0,14737038.0,6277.0,NaN,NaN,41152800.0,-2861.0,7631.0,NaN,10446634.0
327,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1650134001,2022-04-16 18:33:21.000000,2020-01-01,2020-12-31,NaN,NaN,NaN,8390951.0,NaN,539.0,NaN,NaN,557893.0,NaN,NaN,27333565.0,NaN,8390951.0,2200.0,11703273.0,NaN,NaN,NaN,2754429.0,11624247.0,7.0,NaN,NaN,38957812.0,79026.0,77372.0,NaN,8390951.0
638,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1681324161,2023-04-12 18:29:21.000000,2022-01-01,2022-12-31,NaN,NaN,NaN,10008065.0,NaN,12231.0,NaN,NaN,10175.0,NaN,NaN,27047461.0,NaN,10008065.0,2200.0,14516585.0,NaN,NaN,NaN,4498345.0,14352988.0,21639.0,NaN,NaN,41400449.0,163597.0,195267.0,NaN,10008065.0
639,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1681324161,2023-04-12 18:29:21.000000,2021-01-01,2021-12-31,NaN,NaN,NaN,10446634.0,NaN,6780.0,NaN,NaN,133175.0,NaN,NaN,26415762.0,NaN,10446634.0,2565.0,14734177.0,NaN,NaN,NaN,4154368.0,14737038.0,6277.0,NaN,NaN,41152800.0,-2861.0,7631.0,NaN,10446634.0
1460,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1712936573,2024-04-12 15:42:53.000000,2023-01-01,2023-12-31,NaN,NaN,NaN,14630430.0,NaN,NaN,NaN,NaN,317638.0,NaN,NaN,27642181.0,NaN,14630430.0,2700.0,19648588.0,NaN,NaN,NaN,4700520.0,19048477.0,4000.0,NaN,NaN,46690658.0,600111.0,601411.0,NaN,14630430.0
1461,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1712936573,2024-04-12 15:42:53.000000,2022-01-01,2022-12-31,NaN,NaN,NaN,10008065.0,NaN,12231.0,NaN,NaN,10175.0,NaN,NaN,27047461.0,NaN,10008065.0,2200.0,14516585.0,NaN,NaN,NaN,4498345.0,14352988.0,21639.0,NaN,NaN,41400449.0,163597.0,195267.0,NaN,10008065.0
1837,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1745016653,2025-04-18 22:50:53.000000,2024-01-01,2024-12-31,NaN,NaN,NaN,17978665.0,NaN,2700.0,NaN,NaN,203105.0,NaN,NaN,27951556.0,NaN,17978665.0,50248.0,25258575.0,NaN,NaN,NaN,7076805.0,24391749.0,6296.0,NaN,NaN,52343305.0,866826.0,825574.0,NaN,17978665.0
1838,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1745016653,2025-04-18 22:50:53.000000,2023-01-01,2023-12-31,NaN,NaN,NaN,14630430.0,NaN,NaN,NaN,NaN,317638.0,NaN,NaN,27642181.0,NaN,14630430.0,2700.0,19648588.0,NaN,NaN,NaN,4700520.0,19048477.0,4000.0,NaN,NaN,46690658.0,600111.0,601411.0,NaN,14630430.0


Yellowstone Pipe Line Company's entity id is **C001041**. We can now use this ID to filter the DataFrame for YPL.

In [11]:
# Use YPL's entity id to filter the DataFrame
id_ypl = 'C001041'
df_is_ypl = df_is[df_is['entity_id'] == id_ypl]

In [12]:
df_is_ypl

,entity_id,filing_name,publication_time,start_date,end_date,provision_for_deferred_taxes_extraordinary_items,extraordinary_items_net,equity_in_earnings_losses_of_affiliated_companies_including_dividend_income,net_income_loss,income_taxes_on_extraordinary_items,interest_expense,gain_loss_from_disposition_of_discontinued_segments_less_applicable_income_taxes,extraordinary_items_and_accounting_changes,provision_for_deferred_taxes,income_loss_from_operations_of_discontinued_segments_less_applicable_income_taxes,dividend_income_equity_investments,operating_expenses,undistributed_earnings_losses,income_loss_from_continuing_operations,miscellaneous_income,ordinary_income_before_federal_income_taxes,cumulative_effect_of_changes_in_accounting_principles_less_applicable_income_taxes,unusual_or_infrequent_items_credit,income_loss_from_discontinued_operations,federal_income_taxes_on_income_from_continuing_operations,net_carrier_operating_income,miscellaneous_income_charges,income_net_from_noncarrier_property,unusual_or_infrequent_items_debit,operating_revenues,other_income_and_deductions,interest_and_dividend_income,extraordinary_items,income_loss_before_extraordinary_items
326,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1650134001,2022-04-16 18:33:21.000000,2021-01-01,2021-12-31,NaN,NaN,NaN,10446634.0,NaN,6780.0,NaN,NaN,133175.0,NaN,NaN,26415762.0,NaN,10446634.0,2565.0,14734177.0,NaN,NaN,NaN,4154368.0,14737038.0,6277.0,NaN,NaN,41152800.0,-2861.0,7631.0,NaN,10446634.0
327,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1650134001,2022-04-16 18:33:21.000000,2020-01-01,2020-12-31,NaN,NaN,NaN,8390951.0,NaN,539.0,NaN,NaN,557893.0,NaN,NaN,27333565.0,NaN,8390951.0,2200.0,11703273.0,NaN,NaN,NaN,2754429.0,11624247.0,7.0,NaN,NaN,38957812.0,79026.0,77372.0,NaN,8390951.0
638,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1681324161,2023-04-12 18:29:21.000000,2022-01-01,2022-12-31,NaN,NaN,NaN,10008065.0,NaN,12231.0,NaN,NaN,10175.0,NaN,NaN,27047461.0,NaN,10008065.0,2200.0,14516585.0,NaN,NaN,NaN,4498345.0,14352988.0,21639.0,NaN,NaN,41400449.0,163597.0,195267.0,NaN,10008065.0
639,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1681324161,2023-04-12 18:29:21.000000,2021-01-01,2021-12-31,NaN,NaN,NaN,10446634.0,NaN,6780.0,NaN,NaN,133175.0,NaN,NaN,26415762.0,NaN,10446634.0,2565.0,14734177.0,NaN,NaN,NaN,4154368.0,14737038.0,6277.0,NaN,NaN,41152800.0,-2861.0,7631.0,NaN,10446634.0
1460,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1712936573,2024-04-12 15:42:53.000000,2023-01-01,2023-12-31,NaN,NaN,NaN,14630430.0,NaN,NaN,NaN,NaN,317638.0,NaN,NaN,27642181.0,NaN,14630430.0,2700.0,19648588.0,NaN,NaN,NaN,4700520.0,19048477.0,4000.0,NaN,NaN,46690658.0,600111.0,601411.0,NaN,14630430.0
1461,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1712936573,2024-04-12 15:42:53.000000,2022-01-01,2022-12-31,NaN,NaN,NaN,10008065.0,NaN,12231.0,NaN,NaN,10175.0,NaN,NaN,27047461.0,NaN,10008065.0,2200.0,14516585.0,NaN,NaN,NaN,4498345.0,14352988.0,21639.0,NaN,NaN,41400449.0,163597.0,195267.0,NaN,10008065.0
1837,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1745016653,2025-04-18 22:50:53.000000,2024-01-01,2024-12-31,NaN,NaN,NaN,17978665.0,NaN,2700.0,NaN,NaN,203105.0,NaN,NaN,27951556.0,NaN,17978665.0,50248.0,25258575.0,NaN,NaN,NaN,7076805.0,24391749.0,6296.0,NaN,NaN,52343305.0,866826.0,825574.0,NaN,17978665.0
1838,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1745016653,2025-04-18 22:50:53.000000,2023-01-01,2023-12-31,NaN,NaN,NaN,14630430.0,NaN,NaN,NaN,NaN,317638.0,NaN,NaN,27642181.0,NaN,14630430.0,2700.0,19648588.0,NaN,NaN,NaN,4700520.0,19048477.0,4000.0,NaN,NaN,46690658.0,600111.0,601411.0,NaN,14630430.0


From reviewing the above there will be overlap of data due to the fact that for each FERC form there is current and prior year data reported. This results in prior year data being duplicated. For this section of the analyis our goal is to match the current year FERC Form 6, so we will further filter the data to the most recent form. To do this we can filter using the **publication_time** column and only keep rows published in 2025.

In [13]:
# Filter the DataFrame by publication_time
df_is_ypl_25 = df_is_ypl[df_is_ypl.loc[:, 'publication_time'].str.contains('2025-')]

In [14]:
df_is_ypl_25

,entity_id,filing_name,publication_time,start_date,end_date,provision_for_deferred_taxes_extraordinary_items,extraordinary_items_net,equity_in_earnings_losses_of_affiliated_companies_including_dividend_income,net_income_loss,income_taxes_on_extraordinary_items,interest_expense,gain_loss_from_disposition_of_discontinued_segments_less_applicable_income_taxes,extraordinary_items_and_accounting_changes,provision_for_deferred_taxes,income_loss_from_operations_of_discontinued_segments_less_applicable_income_taxes,dividend_income_equity_investments,operating_expenses,undistributed_earnings_losses,income_loss_from_continuing_operations,miscellaneous_income,ordinary_income_before_federal_income_taxes,cumulative_effect_of_changes_in_accounting_principles_less_applicable_income_taxes,unusual_or_infrequent_items_credit,income_loss_from_discontinued_operations,federal_income_taxes_on_income_from_continuing_operations,net_carrier_operating_income,miscellaneous_income_charges,income_net_from_noncarrier_property,unusual_or_infrequent_items_debit,operating_revenues,other_income_and_deductions,interest_and_dividend_income,extraordinary_items,income_loss_before_extraordinary_items
1837,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1745016653,2025-04-18 22:50:53.000000,2024-01-01,2024-12-31,NaN,NaN,NaN,17978665.0,NaN,2700.0,NaN,NaN,203105.0,NaN,NaN,27951556.0,NaN,17978665.0,50248.0,25258575.0,NaN,NaN,NaN,7076805.0,24391749.0,6296.0,NaN,NaN,52343305.0,866826.0,825574.0,NaN,17978665.0
1838,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1745016653,2025-04-18 22:50:53.000000,2023-01-01,2023-12-31,NaN,NaN,NaN,14630430.0,NaN,NaN,NaN,NaN,317638.0,NaN,NaN,27642181.0,NaN,14630430.0,2700.0,19648588.0,NaN,NaN,NaN,4700520.0,19048477.0,4000.0,NaN,NaN,46690658.0,600111.0,601411.0,NaN,14630430.0


From a quick review this does appear to be the correct information. However, the column orders do not match to the FERC Form 6. To be able to complete a full comparison of the data the columns are going to need to be rearanged in an order that matches to the Form 6.

In [15]:
# Get a list of sorted column names
column_names = df_is_ypl_25.columns
column_names = column_names.sort_values()

In [16]:
# Print sorted list
for column in column_names:
    print(column)

cumulative_effect_of_changes_in_accounting_principles_less_applicable_income_taxes
dividend_income_equity_investments
end_date
entity_id
equity_in_earnings_losses_of_affiliated_companies_including_dividend_income
extraordinary_items
extraordinary_items_and_accounting_changes
extraordinary_items_net
federal_income_taxes_on_income_from_continuing_operations
filing_name
gain_loss_from_disposition_of_discontinued_segments_less_applicable_income_taxes
income_loss_before_extraordinary_items
income_loss_from_continuing_operations
income_loss_from_discontinued_operations
income_loss_from_operations_of_discontinued_segments_less_applicable_income_taxes
income_net_from_noncarrier_property
income_taxes_on_extraordinary_items
interest_and_dividend_income
interest_expense
miscellaneous_income
miscellaneous_income_charges
net_carrier_operating_income
net_income_loss
operating_expenses
operating_revenues
ordinary_income_before_federal_income_taxes
other_income_and_deductions
provision_for_deferred_ta

This sorted list was then used to compare to the XBRL form 6 available from the FERC website for Yellowstone Pipe Line Company. The columns match to a specific xbrl tag that correlates with an item in the statements. For example the below screenshot shows the xbrl tag for the first item in the statement, operating revenues:

![](img/ypl-form6-xbrl-tag.png)

The XBRL tag "OperatingReveneus" correlates with column "operating_revenues" in the PUDL database. \
\
By looking up the tag and matching to the list of column names, a list was put together that allowed the database to be sorted in the same order as the Income Statement.

In [17]:
# Produce a printout of the DataFrame in an order that matches to the Income Statement
df_is_ypl_25.loc[:, [
    'publication_time',
    'start_date',
    'end_date',
    'operating_revenues', 
    'operating_expenses',
    'net_carrier_operating_income',
    'other_income_and_deductions',
    'income_net_from_noncarrier_property',
    'interest_and_dividend_income',
    'miscellaneous_income',
    'unusual_or_infrequent_items_credit',
    'interest_expense',
    'miscellaneous_income_charges',
    'unusual_or_infrequent_items_debit',
    'dividend_income_equity_investments',
    'undistributed_earnings_losses',
    'equity_in_earnings_losses_of_affiliated_companies_including_dividend_income',
    'other_income_and_deductions',
    'ordinary_income_before_federal_income_taxes',
    'federal_income_taxes_on_income_from_continuing_operations',
    'provision_for_deferred_taxes',
    'income_loss_from_continuing_operations',
    'income_loss_from_operations_of_discontinued_segments_less_applicable_income_taxes',
    'gain_loss_from_disposition_of_discontinued_segments_less_applicable_income_taxes',
    'income_loss_from_discontinued_operations',
    'income_loss_before_extraordinary_items',
    'extraordinary_items_net',
    'income_taxes_on_extraordinary_items',
    'provision_for_deferred_taxes_extraordinary_items',
    'extraordinary_items',
    'cumulative_effect_of_changes_in_accounting_principles_less_applicable_income_taxes',
    'extraordinary_items_and_accounting_changes',
    'net_income_loss'
    ]]

,publication_time,start_date,end_date,operating_revenues,operating_expenses,net_carrier_operating_income,other_income_and_deductions,income_net_from_noncarrier_property,interest_and_dividend_income,miscellaneous_income,unusual_or_infrequent_items_credit,interest_expense,miscellaneous_income_charges,unusual_or_infrequent_items_debit,dividend_income_equity_investments,undistributed_earnings_losses,equity_in_earnings_losses_of_affiliated_companies_including_dividend_income,other_income_and_deductions,ordinary_income_before_federal_income_taxes,federal_income_taxes_on_income_from_continuing_operations,provision_for_deferred_taxes,income_loss_from_continuing_operations,income_loss_from_operations_of_discontinued_segments_less_applicable_income_taxes,gain_loss_from_disposition_of_discontinued_segments_less_applicable_income_taxes,income_loss_from_discontinued_operations,income_loss_before_extraordinary_items,extraordinary_items_net,income_taxes_on_extraordinary_items,provision_for_deferred_taxes_extraordinary_items,extraordinary_items,cumulative_effect_of_changes_in_accounting_principles_less_applicable_income_taxes,extraordinary_items_and_accounting_changes,net_income_loss
1837,2025-04-18 22:50:53.000000,2024-01-01,2024-12-31,52343305.0,27951556.0,24391749.0,866826.0,NaN,825574.0,50248.0,NaN,2700.0,6296.0,NaN,NaN,NaN,NaN,866826.0,25258575.0,7076805.0,203105.0,17978665.0,NaN,NaN,NaN,17978665.0,NaN,NaN,NaN,NaN,NaN,NaN,17978665.0
1838,2025-04-18 22:50:53.000000,2023-01-01,2023-12-31,46690658.0,27642181.0,19048477.0,600111.0,NaN,601411.0,2700.0,NaN,NaN,4000.0,NaN,NaN,NaN,NaN,600111.0,19648588.0,4700520.0,317638.0,14630430.0,NaN,NaN,NaN,14630430.0,NaN,NaN,NaN,NaN,NaN,NaN,14630430.0


Comparing the sorted DataFrame to the FERC Form 2 for Yellowstone Pipe Line Company does materially match.

## Income Statement Extract Function

With the correct SQL table and the order of columns identified a function can be developed to extract the Income Statement for each entity id.

In [18]:
# Define the Income Statement function
def get_income_statement(db_file, subject_id):

    # Connect to the sql database using sqlite3
    conn = sqlite3.connect(db_file)

    # Query the database file and assign to pandas dataframe
    sql_table = 'income_statement_114_duration'
    sql_query = f'SELECT * FROM {sql_table}'
    df = pd.read_sql(sql_query + " WHERE entity_id = ?", conn, params=(subject_id,))
    
    # Close the database connection
    conn.close()
    
    # Rearange columns to match income statement format
    income_statement_items = (
        'entity_id',
        'filing_name',
        'publication_time',
        'start_date',
        'end_date',
        'operating_revenues', 
        'operating_expenses',
        'net_carrier_operating_income',
        'other_income_and_deductions',
        'income_net_from_noncarrier_property',
        'interest_and_dividend_income',
        'miscellaneous_income',
        'unusual_or_infrequent_items_credit',
        'interest_expense',
        'miscellaneous_income_charges',
        'unusual_or_infrequent_items_debit',
        'dividend_income_equity_investments',
        'undistributed_earnings_losses',
        'equity_in_earnings_losses_of_affiliated_companies_including_dividend_income',
        'other_income_and_deductions',
        'ordinary_income_before_federal_income_taxes',
        'federal_income_taxes_on_income_from_continuing_operations',
        'provision_for_deferred_taxes',
        'income_loss_from_continuing_operations',
        'income_loss_from_operations_of_discontinued_segments_less_applicable_income_taxes',
        'gain_loss_from_disposition_of_discontinued_segments_less_applicable_income_taxes',
        'income_loss_from_discontinued_operations',
        'income_loss_before_extraordinary_items',
        'extraordinary_items_net',
        'income_taxes_on_extraordinary_items',
        'provision_for_deferred_taxes_extraordinary_items',
        'extraordinary_items',
        'cumulative_effect_of_changes_in_accounting_principles_less_applicable_income_taxes',
        'extraordinary_items_and_accounting_changes',
        'net_income_loss'
    )
    
    df = df.loc[:, income_statement_items]
    
    return df

We now can test the function using Yellowstone Pipe Line Company's entity id "C001041".

In [19]:
# Set function parameters
db_file = 'ferc6_xbrl.sqlite'
subject_id = 'C001041'

In [20]:
# Query the PUDL SQL Database
df_ypl = get_income_statement(db_file, subject_id)

In [21]:
df_ypl

,entity_id,filing_name,publication_time,start_date,end_date,operating_revenues,operating_expenses,net_carrier_operating_income,other_income_and_deductions,income_net_from_noncarrier_property,interest_and_dividend_income,miscellaneous_income,unusual_or_infrequent_items_credit,interest_expense,miscellaneous_income_charges,unusual_or_infrequent_items_debit,dividend_income_equity_investments,undistributed_earnings_losses,equity_in_earnings_losses_of_affiliated_companies_including_dividend_income,other_income_and_deductions,ordinary_income_before_federal_income_taxes,federal_income_taxes_on_income_from_continuing_operations,provision_for_deferred_taxes,income_loss_from_continuing_operations,income_loss_from_operations_of_discontinued_segments_less_applicable_income_taxes,gain_loss_from_disposition_of_discontinued_segments_less_applicable_income_taxes,income_loss_from_discontinued_operations,income_loss_before_extraordinary_items,extraordinary_items_net,income_taxes_on_extraordinary_items,provision_for_deferred_taxes_extraordinary_items,extraordinary_items,cumulative_effect_of_changes_in_accounting_principles_less_applicable_income_taxes,extraordinary_items_and_accounting_changes,net_income_loss
0,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1650134001,2022-04-16 18:33:21.000000,2021-01-01,2021-12-31,41152800.0,26415762.0,14737038.0,-2861.0,None,7631.0,2565.0,None,6780.0,6277.0,None,None,None,None,-2861.0,14734177.0,4154368.0,133175.0,10446634.0,None,None,None,10446634.0,None,None,None,None,None,None,10446634.0
1,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1650134001,2022-04-16 18:33:21.000000,2020-01-01,2020-12-31,38957812.0,27333565.0,11624247.0,79026.0,None,77372.0,2200.0,None,539.0,7.0,None,None,None,None,79026.0,11703273.0,2754429.0,557893.0,8390951.0,None,None,None,8390951.0,None,None,None,None,None,None,8390951.0
2,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1681324161,2023-04-12 18:29:21.000000,2022-01-01,2022-12-31,41400449.0,27047461.0,14352988.0,163597.0,None,195267.0,2200.0,None,12231.0,21639.0,None,None,None,None,163597.0,14516585.0,4498345.0,10175.0,10008065.0,None,None,None,10008065.0,None,None,None,None,None,None,10008065.0
3,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1681324161,2023-04-12 18:29:21.000000,2021-01-01,2021-12-31,41152800.0,26415762.0,14737038.0,-2861.0,None,7631.0,2565.0,None,6780.0,6277.0,None,None,None,None,-2861.0,14734177.0,4154368.0,133175.0,10446634.0,None,None,None,10446634.0,None,None,None,None,None,None,10446634.0
4,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1712936573,2024-04-12 15:42:53.000000,2023-01-01,2023-12-31,46690658.0,27642181.0,19048477.0,600111.0,None,601411.0,2700.0,None,NaN,4000.0,None,None,None,None,600111.0,19648588.0,4700520.0,317638.0,14630430.0,None,None,None,14630430.0,None,None,None,None,None,None,14630430.0
5,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1712936573,2024-04-12 15:42:53.000000,2022-01-01,2022-12-31,41400449.0,27047461.0,14352988.0,163597.0,None,195267.0,2200.0,None,12231.0,21639.0,None,None,None,None,163597.0,14516585.0,4498345.0,10175.0,10008065.0,None,None,None,10008065.0,None,None,None,None,None,None,10008065.0
6,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1745016653,2025-04-18 22:50:53.000000,2024-01-01,2024-12-31,52343305.0,27951556.0,24391749.0,866826.0,None,825574.0,50248.0,None,2700.0,6296.0,None,None,None,None,866826.0,25258575.0,7076805.0,203105.0,17978665.0,None,None,None,17978665.0,None,None,None,None,None,None,17978665.0
7,C001041,Yellowstone_Pipe_Line_Company_form6_Q4_1745016653,2025-04-18 22:50:53.000000,2023-01-01,2023-12-31,46690658.0,27642181.0,19048477.0,600111.0,None,601411.0,2700.0,None,NaN,4000.0,None,None,None,None,600111.0,19648588.0,4700520.0,317638.0,14630430.0,None,None,None,14630430.0,None,None,None,None,None,None,14630430.0


## Extract Income Statement for Other Entities

This function can now be tested with additional entities to ensure it works with different entities. It now will be tested with Phillips 66 Carrier Pipeline and Phillips 66 Pipeline.

This function can now be tested with other Montana oil pipelines. The sql database was reviewed to determine the entity_id's that match to the remaining Montana oil pipelines. Two of the additional pipelines listed below will now be tested:
- Phillips 66 Carrier LLC = Entity ID: C003494 
- Phillips 66 Pipeline LLC = Entity ID: C003013

In [22]:
# Assign additional entity_id's to variables
id_p66_carrier = 'C003494'
id_p66_pieline = 'C003013'

In [23]:
# Extract additional income statements using entitiy_id's
df_p66_carrier = get_income_statement(db_file, id_p66_carrier)
df_p66_pipeline = get_income_statement(db_file, id_p66_pieline)

In [24]:
df_p66_carrier

,entity_id,filing_name,publication_time,start_date,end_date,operating_revenues,operating_expenses,net_carrier_operating_income,other_income_and_deductions,income_net_from_noncarrier_property,interest_and_dividend_income,miscellaneous_income,unusual_or_infrequent_items_credit,interest_expense,miscellaneous_income_charges,unusual_or_infrequent_items_debit,dividend_income_equity_investments,undistributed_earnings_losses,equity_in_earnings_losses_of_affiliated_companies_including_dividend_income,other_income_and_deductions,ordinary_income_before_federal_income_taxes,federal_income_taxes_on_income_from_continuing_operations,provision_for_deferred_taxes,income_loss_from_continuing_operations,income_loss_from_operations_of_discontinued_segments_less_applicable_income_taxes,gain_loss_from_disposition_of_discontinued_segments_less_applicable_income_taxes,income_loss_from_discontinued_operations,income_loss_before_extraordinary_items,extraordinary_items_net,income_taxes_on_extraordinary_items,provision_for_deferred_taxes_extraordinary_items,extraordinary_items,cumulative_effect_of_changes_in_accounting_principles_less_applicable_income_taxes,extraordinary_items_and_accounting_changes,net_income_loss
0,C003494,Phillips_66_Carrier_LLC_form6_Q4_1650310849,2022-04-18 19:40:49.000000,2020-01-01,2020-12-31,424828311.0,264941733.0,159886578.0,50584052.0,50176796.0,NaN,435456.0,None,74.0,28126.0,None,None,None,None,50584052.0,210470630.0,None,None,210470630.0,None,None,None,210470630.0,None,None,None,None,None,None,210470630.0
1,C003494,Phillips_66_Carrier_LLC_form6_Q4_1650310849,2022-04-18 19:40:49.000000,2021-01-01,2021-12-31,486569876.0,268653731.0,217916145.0,37730705.0,36879551.0,NaN,869661.0,None,13.0,18494.0,None,None,None,None,37730705.0,255646850.0,None,None,255646850.0,None,None,None,255646850.0,None,None,None,None,None,None,255646850.0
2,C003494,Phillips_66_Carrier_LLC_form6_Q4_1681321057,2023-04-12 17:37:37.000000,2022-01-01,2022-12-31,569108563.0,276517762.0,292590801.0,37180136.0,33081352.0,3936418.0,184314.0,None,NaN,21948.0,None,None,None,None,37180136.0,329770937.0,None,None,329770937.0,None,None,None,329770937.0,None,None,None,None,None,None,329770937.0
3,C003494,Phillips_66_Carrier_LLC_form6_Q4_1681321057,2023-04-12 17:37:37.000000,2021-01-01,2021-12-31,486569876.0,268653731.0,217916145.0,37730705.0,36879551.0,NaN,869661.0,None,13.0,18494.0,None,None,None,None,37730705.0,255646850.0,None,None,255646850.0,None,None,None,255646850.0,None,None,None,None,None,None,255646850.0
4,C003494,Phillips_66_Carrier_LLC_form6_Q4_1712937587,2024-04-12 15:59:47.000000,2023-01-01,2023-12-31,631429929.0,273260536.0,358169393.0,58612084.0,30865131.0,26492330.0,1281928.0,None,NaN,27305.0,None,None,None,None,58612084.0,416781477.0,None,None,416781477.0,None,None,None,416781477.0,None,None,None,None,None,None,416781477.0
5,C003494,Phillips_66_Carrier_LLC_form6_Q4_1712937587,2024-04-12 15:59:47.000000,2022-01-01,2022-12-31,569108563.0,276517762.0,292590801.0,37180136.0,33081352.0,3936418.0,184314.0,None,NaN,21948.0,None,None,None,None,37180136.0,329770937.0,None,None,329770937.0,None,None,None,329770937.0,None,None,None,None,None,None,329770937.0
6,C003494,Phillips_66_Carrier_LLC_form6_Q4_1745014218,2025-04-18 22:10:18.000000,2024-01-01,2024-12-31,672931827.0,266516602.0,406415225.0,90025106.0,47169288.0,42839171.0,42199.0,None,67.0,25485.0,None,None,None,None,90025106.0,496440331.0,None,None,496440331.0,None,None,None,496440331.0,None,None,None,None,None,None,496440331.0
7,C003494,Phillips_66_Carrier_LLC_form6_Q4_1745014218,2025-04-18 22:10:18.000000,2023-01-01,2023-12-31,631429929.0,273260536.0,358169393.0,58612084.0,30865131.0,26492330.0,1281928.0,None,NaN,27305.0,None,None,None,None,58612084.0,416781477.0,None,None,416781477.0,None,None,None,416781477.0,None,None,None,None,None,None,416781477.0


In [25]:
df_p66_pipeline

,entity_id,filing_name,publication_time,start_date,end_date,operating_revenues,operating_expenses,net_carrier_operating_income,other_income_and_deductions,income_net_from_noncarrier_property,interest_and_dividend_income,miscellaneous_income,unusual_or_infrequent_items_credit,interest_expense,miscellaneous_income_charges,unusual_or_infrequent_items_debit,dividend_income_equity_investments,undistributed_earnings_losses,equity_in_earnings_losses_of_affiliated_companies_including_dividend_income,other_income_and_deductions,ordinary_income_before_federal_income_taxes,federal_income_taxes_on_income_from_continuing_operations,provision_for_deferred_taxes,income_loss_from_continuing_operations,income_loss_from_operations_of_discontinued_segments_less_applicable_income_taxes,gain_loss_from_disposition_of_discontinued_segments_less_applicable_income_taxes,income_loss_from_discontinued_operations,income_loss_before_extraordinary_items,extraordinary_items_net,income_taxes_on_extraordinary_items,provision_for_deferred_taxes_extraordinary_items,extraordinary_items,cumulative_effect_of_changes_in_accounting_principles_less_applicable_income_taxes,extraordinary_items_and_accounting_changes,net_income_loss
0,C003013,Phillips_66_Pipeline_LLC_form6_Q4_1650309999,2022-04-18 19:26:39.000000,2021-01-01,2021-12-31,186956983.0,153904710.0,33052273.0,17857360.0,-21850462.0,13351202.0,6603406.0,0.0,6.0,85785.0,0.0,15326991.0,4512014.0,19839005.0,17857360.0,50909633.0,0.0,0.0,50909633.0,0.0,0.0,None,50909633.0,0.0,0.0,0.0,NaN,0.0,NaN,50909633.0
1,C003013,Phillips_66_Pipeline_LLC_form6_Q4_1650309999,2022-04-18 19:26:39.000000,2020-01-01,2020-12-31,158754656.0,174185942.0,-15431286.0,31100455.0,-17245524.0,14151274.0,5982232.0,9185120.0,1.0,145339.0,0.0,19225062.0,-52369.0,19172693.0,31100455.0,15669169.0,0.0,0.0,15669169.0,0.0,0.0,None,15669169.0,0.0,0.0,0.0,0.0,0.0,0.0,15669169.0
2,C003013,Phillips_66_Pipeline_LLC_form6_Q4_1681324041,2023-04-12 18:27:21.000000,2021-01-01,2021-12-31,186956983.0,153904710.0,33052273.0,17857360.0,-21850462.0,13351202.0,6603406.0,NaN,6.0,85785.0,NaN,15326991.0,4512014.0,19839005.0,17857360.0,50909633.0,NaN,NaN,50909633.0,NaN,NaN,None,50909633.0,NaN,NaN,NaN,NaN,NaN,NaN,50909633.0
3,C003013,Phillips_66_Pipeline_LLC_form6_Q4_1681324041,2023-04-12 18:27:21.000000,2022-01-01,2022-12-31,174934355.0,137804508.0,37129847.0,23159344.0,-20599749.0,14201935.0,4083786.0,NaN,50.0,105831.0,NaN,8785901.0,16793352.0,25579253.0,23159344.0,60289191.0,NaN,NaN,60289191.0,NaN,NaN,None,60289191.0,NaN,NaN,NaN,NaN,NaN,NaN,60289191.0
4,C003013,Phillips_66_Pipeline_LLC_form6_Q4_1712937909,2024-04-12 16:05:09.000000,2023-01-01,2023-12-31,143117134.0,107075727.0,36041407.0,32722043.0,-16432648.0,17098780.0,5259863.0,NaN,13.0,796197.0,NaN,11805879.0,15786379.0,27592258.0,32722043.0,68763450.0,NaN,NaN,68763450.0,NaN,NaN,None,68763450.0,NaN,NaN,NaN,NaN,NaN,NaN,68763450.0
5,C003013,Phillips_66_Pipeline_LLC_form6_Q4_1712937909,2024-04-12 16:05:09.000000,2022-01-01,2022-12-31,174934355.0,137804508.0,37129847.0,23159344.0,-20599749.0,14201935.0,4083786.0,NaN,50.0,105831.0,NaN,8785901.0,16793352.0,25579253.0,23159344.0,60289191.0,NaN,NaN,60289191.0,NaN,NaN,None,60289191.0,NaN,NaN,NaN,NaN,NaN,NaN,60289191.0
6,C003013,Phillips_66_Pipeline_LLC_form6_Q4_1745012446,2025-04-18 21:40:46.000000,2024-01-01,2024-12-31,95864006.0,75369658.0,20494348.0,11384496.0,-29767622.0,NaN,4854377.0,NaN,3.0,63761.0,NaN,29194428.0,7167077.0,36361505.0,11384496.0,31878844.0,NaN,NaN,31878844.0,NaN,NaN,None,31878844.0,NaN,NaN,NaN,NaN,NaN,NaN,31878844.0
7,C003013,Phillips_66_Pipeline_LLC_form6_Q4_1745012446,2025-04-18 21:40:46.000000,2023-01-01,2023-12-31,143117134.0,107075727.0,36041407.0,32722043.0,-16432648.0,17098780.0,5259863.0,NaN,13.0,796197.0,NaN,11805879.0,15786379.0,27592258.0,32722043.0,68763450.0,NaN,NaN,68763450.0,NaN,NaN,None,68763450.0,NaN,NaN,NaN,NaN,NaN,NaN,68763450.0


## Conclusion

We were able to successfully extract a complete FERC Form 6 Income Statement from the PUDL SQL database for multiple pipelines and were able to build a custom extract function. This function can now be used to extract Form 6 Income Statement data as needed.